In [91]:
from keras_preprocessing.image import load_img
!pip install keras-preprocessing


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [92]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os 
import pandas as pd
import numpy as np

In [93]:
TRAIN_DIR = "D:\\intern\\emotion detection\\images\\train"
TEST_DIR = "D:\\intern\\emotion detection\\images\\test"

In [94]:
def createdataframe (dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir (os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [95]:
train = pd.DataFrame()
train ["D:\\intern\\emotion detection\\images\\train"], train['label'] =createdataframe (TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [96]:
print(train)

                D:\intern\emotion detection\images\train     label
0      D:\intern\emotion detection\images\train\angry...     angry
1      D:\intern\emotion detection\images\train\angry...     angry
2      D:\intern\emotion detection\images\train\angry...     angry
3      D:\intern\emotion detection\images\train\angry...     angry
4      D:\intern\emotion detection\images\train\angry...     angry
...                                                  ...       ...
28814  D:\intern\emotion detection\images\train\surpr...  surprise
28815  D:\intern\emotion detection\images\train\surpr...  surprise
28816  D:\intern\emotion detection\images\train\surpr...  surprise
28817  D:\intern\emotion detection\images\train\surpr...  surprise
28818  D:\intern\emotion detection\images\train\surpr...  surprise

[28819 rows x 2 columns]


In [97]:
test = pd.DataFrame()
test ["D:\\intern\\emotion detection\\images\\test"], test['label'] =createdataframe (TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [98]:
print(test)


                D:\intern\emotion detection\images\test     label
0     D:\intern\emotion detection\images\test\angry\...     angry
1     D:\intern\emotion detection\images\test\angry\...     angry
2     D:\intern\emotion detection\images\test\angry\...     angry
3     D:\intern\emotion detection\images\test\angry\...     angry
4     D:\intern\emotion detection\images\test\angry\...     angry
...                                                 ...       ...
7061  D:\intern\emotion detection\images\test\surpri...  surprise
7062  D:\intern\emotion detection\images\test\surpri...  surprise
7063  D:\intern\emotion detection\images\test\surpri...  surprise
7064  D:\intern\emotion detection\images\test\surpri...  surprise
7065  D:\intern\emotion detection\images\test\surpri...  surprise

[7066 rows x 2 columns]


In [99]:
from tqdm import tqdm


In [100]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale = True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features
    
    

In [101]:
train_features = extract_features (train["D:\\intern\\emotion detection\\images\\train"])

  0%|                                                                                                                      | 0/28819 [00:00<?, ?it/s]C:\Users\gokil\anaconda3\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 28819/28819 [00:27<00:00, 1049.73it/s]


In [102]:
test_features = extract_features (test ["D:\\intern\\emotion detection\\images\\test"])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 7066/7066 [00:06<00:00, 1082.07it/s]


In [103]:
x_train = train_features/255.0
x_test = test_features/255.0

In [104]:
from sklearn.preprocessing import LabelEncoder

In [105]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [106]:
    y_train = le.transform(train['label'])
    y_test = le.transform(test['label'])

In [107]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [108]:
model = Sequential()

# Convolutional Layers
model.add(Conv2D(128, kernel_size=(3, 3), activation="relu", input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully connected layers
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7, activation="softmax"))  

C:\Users\gokil\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [109]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model.fit(x_train, y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test), callbacks=[early_stopping])


Epoch 1/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 530s 2s/step - accuracy: 0.2350 - loss: 1.8378 - val_accuracy: 0.3068 - val_loss: 1.7232
Epoch 2/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 1343s 6s/step - accuracy: 0.3143 - loss: 1.6922 - val_accuracy: 0.4226 - val_loss: 1.4971
Epoch 3/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 693s 3s/step - accuracy: 0.4084 - loss: 1.5090 - val_accuracy: 0.4479 - val_loss: 1.4423
Epoch 4/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 681s 3s/step - accuracy: 0.4507 - loss: 1.4277 - val_accuracy: 0.4999 - val_loss: 1.3038
Epoch 5/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 695s 3s/step - accuracy: 0.4789 - loss: 1.3600 - val_accuracy: 0.5214 - val_loss: 1.2514
Epoch 6/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 766s 3s/step - accuracy: 0.5060 - loss: 1.2966 - val_accuracy: 0.5384 - val_loss: 1.2069
Epoch 7/100
226/226 ━━━━━━━━━━━━━━━━━━━━ 928s 4s/step - accuracy: 0.5161 - loss: 1.2639 - val_accuracy: 0.5511 - val_loss: 1.1867
Epoch 8/100
  5/226 ━━━━━━━━━━━━━━━━━━━━ 21:41 6s/step - accuracy: 0.5032 - loss: 1.2350

In [ ]:

# Convert model to JSON format
model_json = model.to_json()

# Save the model architecture to a JSON file
with open("emotiondetector.json", "w") as json_file:
    json_file.write(model_json)  # Corrected this line

# Save the trained model weights to an HDF5 file
model.save("emotiondetector.keras")


In [ ]:
from keras.models import model_from_json

In [ ]:
from keras.models import model_from_json

# Load the model architecture from the JSON file
json_file = open("emotiondetector.json", "r")
model_json = json_file.read()  # Corrected the variable name
json_file.close()

# Recreate the model from the JSON architecture
model = model_from_json(model_json)  # Corrected this line

# Load the model weights
model.load_weights("emotiondetector.h5")


In [ ]:
label = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [ ]:
from keras.preprocessing.image import load_img
import numpy as np

def ef(image):
    img = load_img(image, color_mode='grayscale')  # Corrected grayscale parameter
    feature = np.array(img)
    feature = feature.reshape(1, 48, 48, 1)  # Applied reshape to 'feature'
    return feature / 255.0  # Normalizing the image data


In [ ]:
import matplotlib.pyplot as plt

Image = "D:\\intern\\emotion detection\\images\\train\\happy\\67.jpg"  # or use double backslashes
print("Original image is happy")
img = ef(Image)  # Preprocess the image
pred = model.predict(img)  # Predict the label for the processed image
pred_label = label[pred.argmax()]  # Map prediction to label
print("Model prediction is:", pred_label)
plt.imshow(img.reshape(48,48),cmap='gray') 